.

Data_Engineering_TIL_(20200225)

학습 시 참고자료(출처) : 


- 블로그 글 : Spark core concepts explained 


- URL : https://luminousmen.com/post/spark-core-concepts-explained


### 1. 개요

스파크 아키텍처는 아래 두가지 메인 개념이 기본이라고 할 수 있다.

- Resilient Distributed Dataset (RDD)


- Directed Acyclic Graph (DAG)

### 2. Resilient Distributed Dataset (RDD)

RDD에는 arbitrary collection of objects 를 저장할 수 있다. RDD에 저장된 각각의 데이터셋은 논리적으로 클러스터 노드들에 분산되어 있기 때문에 parallel하게 연산이 가능하다.

물리적으로는 RDD는 JVM이라는 드라이버 객체에 저장된다. RDD에 데이터가 저장되는 양상은 HDFS, Hbase, Cassandra 등 persisted한 데이터 스토어에 저장되어 있거나 캐시형태(memory, memory+disks, disk only, etc.) 또는 다른형태의 RDD에 저장된다고 할 수 있다.

RDD는 아래와 같은 메타데이터를 저장한다.

1) Partitions 

- 데이터 셋이 split된 형태로 클러스터 노드들 전반에 위치하게 된다.


- 하나의 파티션은 각각의 클러스터 노드에서 연산되는 미니멀한 데이터 배치형태다.


2) Dependencies 

- lineage graph라고 알고 있는 개념


- Computation과 관련있는 parent(부모) RDD 들의 리스트

3) Computation 

- Dependencies에서 parent RDD에게 주어진 child(자식) RDD를 연산하는 기능


4) Preferred Locations 

- put computations on partitions (data locality) 하기 최적의 위치 


5) Partitioner 

- 데이터가 어떻게 partitions로 split되는지(디폴트로 HashPartitioner로 split한다)


RDD는 lineage graph에 저장되어 그 RDD가 어떻게 creat 되었는지 알기 때문에 RDD는 recreate 될 수 있다.

RDD also can be materialized, in memory or on disk.

Example :

In [1]:
rdd = sc.parallelize(range(20))  # create RDD

rdd

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1582606619430_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[1] at RDD at PythonRDD.scala:53

In [2]:
rdd.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

RDD는 또한 메모리에 캐싱될 수 있고 manually partitioned 될 수 있다. RDD를 자주 호출할 수록 캐싱이 유리하다. 반면에 그만큼 다른 computations에 대한 자원이 뺏길 수 있다는 점은 고려해야 한다.

그리고 manual partitioning은 partition들 간에 데이터 밸런스를 올바로 맞춰준다는 점에서 중요하다. 일반적으로 파티션을 작게 잡을 수록 데이터들이 좀 더 동일하게 분배가 된다. 따라서 파티션을 작은수로 잡을수록 다수의 repartition(data reorganizations during computations)들의 task들을 좀 더 boost할 수 있다. 

파티션 수에 따른 데이터양을 확인해보자

In [3]:
# get current number of paritions
rdd.getNumPartitions()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [4]:
# collect data on driver based on partitions
rdd.glom().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]

일반적으로 spark application는 아래와 같은 과정으로 동작하게 된다.

먼저 RDD를 생성한다.(for example, we set data source as file on HDFS)

그리고 그것을 map,reduce,join 등을 이용해서 transform한다.

HDFS에 다시 저장해주는 등 결과를 처리해준다.

![1](https://user-images.githubusercontent.com/41605276/75301191-74165b80-587d-11ea-9adb-a3397fb88cbc.png)

RDD로 두가지 타입의 operation을 할 수 있다.

### 3. Transformations

transformation 연산을 RDD에 적용한 결과는 새로운 RDD가 생성된다. 주어진 데이터에 대해 어느정도 convert를 해주는 연산이라고 할 수 있다.

transformation 연산은 lazy하게 이루어진다. 다시말해서 연산을 명령해도 바로 execute를 하는 것이 아니다. spark은 DAG로부터 연산이 call 되게 하는 레코드를 유지하고 있다.


spark RDD를 우리가 임의로 transformation 할 수 있는 데이터로 생각할 수 있다. lazy 특성때문에 action 연산을 언제든지 콜헤사 연산할 수 있다. 따라서 필요할때만 연산이 이루어지는 것이다. 이는 low level 관점 최적화를 이룰 수 있는 여지를 준다.

higi level 단에서는 두가지 그룹의 transformation을 RDD에 적용할 수 있다. 하나는 narrow transformation이고, 다른 하나는 wide transformation이다.

![22](https://user-images.githubusercontent.com/41605276/75324721-e6596100-58ba-11ea-823c-bbcafe75c9bd.png)

Narrow trasformation은 데이터가 셔플되거나 partition들이 서로 reorganize 되지 않는다. 예를 들어서 map, filter 같은 연산들을 말한다. Narrow trasformation은 single stage에서 서로 grouping되거나 pipelining된다.

데이터가 partitinon간에 rearrange가 일어날때 shuffle이 발생했다고 할 수 있다. transformation는 어떤 컬럼에서 value 값들의 summaize하는거 같은 다른 partition들의 information이 요구되어 지는데 이런게 필요한 경우 발생한다.

spark는 각각의 partition과 그것을 새로운 partition으로 combine하는 것(다른 excutor에서 연산되는 것들)들로부터 데이터를 gathering 할 것이다.

But there are exceptions, operations like coalesce may cause the task to work with multiple input partitions, but the transformation will still be considered narrow because the input records used to compute any output record can still be found only in a limited subset of partitions.

그러나 큰덩어리로 합치는 연산은 task를 multiple한 input partition으로 working하기 때문에 이런점에서 예외는 있다. 그러나 trasnformation은 narrow로 될 것이다. 왜냐하면 input record들이 어떤 output record가 되던지간에 여전히 partition들의 제한적인 subset이기 때문이다.

filter transformation 연산을 실습해보자

In [2]:
rdd = sc.parallelize(range(20))
filteredRDD = rdd.filter(lambda x: x > 10)

## to see the execution graph; only one stage is created
print(filteredRDD.toDebugString())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b'(2) PythonRDD[1] at RDD at PythonRDD.scala:53 []\n |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195 []'

In [3]:
filteredRDD.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[11, 12, 13, 14, 15, 16, 17, 18, 19]

위에 예시에서 우리는 shuffle 연산을 하지 않았고, 각각의 파티션이 독립적으로 연산된것을 할 수 있다.

그러나 spark는 groupByKey, reduceByKey 연산 같이 transformations with wide dependencies(namely wide transformations)도 지원하고 있다. 각각 dependency한 연산이라고 할 수 있고, 데이터들이 부모 RDD 아래의 여러 partition들로 위치해서 연산된다. 같은 key의 모든 데이터는 같은 partition에서 single task로 연산되어야만 한다. 이런 연산들을 수행하기 위해서 spark는 shuffling을 해줘야하고, 데이터들이 분산된 클러스터 노드들로 moving 시켜줘야하며, 아래 예시와 같이 새로운 형태의 partition set을 새로운 stage로 forming 해줘야 한다.

In [4]:
## group data based on mod
groupedRDD = filteredRDD.groupBy(lambda x: x % 2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
## two separate stages are created, because of the shuffle
print(groupedRDD.toDebugString())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b'(2) PythonRDD[6] at RDD at PythonRDD.scala:53 []\n |  MapPartitionsRDD[5] at mapPartitions at PythonRDD.scala:133 []\n |  ShuffledRDD[4] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(2) PairwiseRDD[3] at groupBy at <stdin>:2 []\n    |  PythonRDD[2] at groupBy at <stdin>:2 []\n    |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195 []'

### 4. Actions

Action은 우리가 어떤 연산들의 결과를 도출하고 싶을때 적용한다. 예를들어서 data를 disk에 저장하거나 콘솔상으로 output을 보려고 할때를 말한다. collect 연산은 말그대로 데이터를 collect하는 연산으로 action에 해당한다.

action은 lazy가 아니다. data processing을 실질적으로 trigger한다.

reduce를 이용하여 filtering한 데이터를 sum하여 가져와보자 

In [6]:
filteredRDD.reduce(lambda a, b: a + b)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

135

### 5. DAG

the user has to break down the whole job into smaller jobs and chain them together to go along with MapReduce하는 Hadoop과 다르게 spark는 클러스터상에서 partition된 데이터들을 parallel하게 컴퓨팅하는 task를 정의할 수 있다. 이런 task 정의를 하고, spark는 DAG(Directed Acyclic Graph)라는 논리적인 연산을 수행하는 flow를 RDD partition and the edge is transformation on the data 하는 노드에서 a graph that is directed and acyclic 하는 형태로 구현하게 된다. 그래서 spark는 이런 연산에 대한 plan들을 build하게 된다.

![3](https://user-images.githubusercontent.com/41605276/75401498-abe8d600-5945-11ea-8455-cbc1a8df36ad.png)

DAGScheduler는 task들의 stage를 나누는 오퍼레이터이다. stage는 input data partition들을 베이스로 하는 task들로 이루어져있다. DAGScheduler는 일부 transformation들을 파이프라이닝 한다. 예를들어서 많은 map 오퍼레이터들은 single task로 통합될 수 있다. DAGScheduler의 최종결과는 stage들의 set이다. stage들은 TaskScheduler로 보내진다. 다수의 task들이 partition 갯수에 의존한다. TaskScheduler는 클러스터 매니저를 통해서 테스크를 런칭한다. TaskScheduler는 스테이지들에 대한 의존성을 인식하지 못한다.

RDDs are capable of defining location preference to compute partitions. Location preference는 RDD의 로케이션을 나타내는 것이다. DAGScheduler는 데이터 로컬리티를 최대한 활용한다.(The DAGScheduler places the partitions in such a way that the task is close to data as much as possible(data locality).)